In [ ]:
!pip install torchmetrics

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random
from torchmetrics.classification import Recall, Accuracy, AUROC, Precision

In [ ]:
FILL = "FILL"

In [ ]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2024-12-03 23:16:52--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.12’

smsspamcollection.z     [   <=>              ] 198.65K   308KB/s    in 0.6s    

2024-12-03 23:16:54 (308 KB/s) - ‘smsspamcollection.zip.12’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
!unzip -o smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
!head -10 SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [ ]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2024-12-03 23:16:54--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.13’

smsspamcollection.z     [   <=>              ] 198.65K   308KB/s    in 0.6s    

2024-12-03 23:16:55 (308 KB/s) - ‘smsspamcollection.zip.13’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
file_path = 'SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index = [])
with open(file_path) as f:
    for line in f.readlines():
        split = line.split('\t')
        df = pd.concat([
                df,
                pd.DataFrame.from_dict({
                    'label': [1 if split[0] == 'spam' else 0],
                    'text': [split[1]]
                })
            ],
            ignore_index=True
        )
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
text = df.text.values
labels = df.label.values

In [ ]:
# Set to the GPT2Tokenizer and set lower case to True
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', do_lower_case = True)
tokenizer.pad_token = tokenizer.eos_token
# Set the padding to 'left' or 'right'?
# Remember we want to use the last token's embedding to represent the entire sentence
tokenizer.padding_side = 'left'

In [ ]:
def print_rand_sentence():
    '''Displays the tokens and respective IDs of a random text sample'''
    index = random.randint(0, len(text)-1)
    print(text[index])
    table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

FREE RINGTONE text FIRST to 87131 for a poly or text GET to 87131 for a true tone! Help? 0845 2814032 16 after 1st free, tones are 3x£150pw to e£nd txt stop

╒══════════╤═════════════╕
│ Tokens   │   Token IDs │
╞══════════╪═════════════╡
│ FREE     │       39274 │
├──────────┼─────────────┤
│ ĠR       │         371 │
├──────────┼─────────────┤
│ INGTON   │       17480 │
├──────────┼─────────────┤
│ E        │          36 │
├──────────┼─────────────┤
│ Ġtext    │        2420 │
├──────────┼─────────────┤
│ ĠFIRST   │       31328 │
├──────────┼─────────────┤
│ Ġto      │         284 │
├──────────┼─────────────┤
│ Ġ87      │       10083 │
├──────────┼─────────────┤
│ 131      │       22042 │
├──────────┼─────────────┤
│ Ġfor     │         329 │
├──────────┼─────────────┤
│ Ġa       │         257 │
├──────────┼─────────────┤
│ Ġpoly    │        7514 │
├──────────┼─────────────┤
│ Ġor      │         393 │
├──────────┼─────────────┤
│ Ġtext    │        2420 │
├──────────┼─────────────┤
│ ĠGE

In [ ]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  # Use the tokenizer and the encode_plus methods to return the right data we'll need
  # Set max_length = 32 and return_tokens = 'pt'
  # Set other fields to the appropriate booleans needed
  return tokenizer.encode_plus(input_text, add_special_tokens = True,
                               max_length = 32, pad_to_max_length = True,
                               return_attention_mask = True, return_tensors = 'pt')

for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])

# Gather all the torch_id, attention masks, and labels
token_id = torch.cat(token_id, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2834: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
def print_rand_sentence_encoding():
    '''Displays tokens, token IDs and attention mask of a random text sample'''
    index = random.randint(0, len(text) - 1)
    tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
    print(tokens)
    token_ids = [i.numpy() for i in token_id[index]]
    attention = [i.numpy() for i in attention_masks[index]]
    table = np.array([tokens, token_ids, attention]).T
    print(
        tabulate(
            table,
            headers = ['Tokens', 'Token IDs', 'Attention Mask'],
            tablefmt = 'fancy_grid')
    )

print_rand_sentence_encoding()

['<|endoftext|>', '<|endoftext|>', 'Anyway', 'Ġseriously', 'Ġhit', 'Ġme', 'Ġup', 'Ġwhen', 'Ġyou', "'re", 'Ġback', 'Ġbecause', 'Ġotherwise', 'ĠI', 'Ġhave', 'Ġto', 'Ġlight', 'Ġup', 'Ġwith', 'Ġarm', 'and', 'Ġand', 'Ġhe', 'Ġalways', 'Ġhas', 'Ġshit', 'Ġand', '/', 'or', 'Ġis', 'Ġvomiting', 'Ċ']
╒═══════════════╤═════════════╤══════════════════╕
│ Tokens        │   Token IDs │   Attention Mask │
╞═══════════════╪═════════════╪══════════════════╡
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ Anyway        │       23795 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġseriously    │        6411 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġhit          │        2277 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġme           │         502 │                

In [ ]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
# Use train_test_split
train_idx, val_idx = train_test_split(np.arange(len(labels)),
                                      test_size=val_ratio,
                                      stratify=labels,
                                      shuffle=True,
                                      random_state=42)


# Train and validation sets
# Set to TensorDataset
train_set = TensorDataset(token_id[train_idx], attention_masks[train_idx], labels[train_idx])

val_set = TensorDataset(token_id[val_idx], attention_masks[val_idx], labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(train_set, sampler=RandomSampler(train_set), batch_size=batch_size)
validation_dataloader = DataLoader(val_set, sampler=SequentialSampler(val_set), batch_size=batch_size)

### Load specific versions of the model

In [ ]:
# Load the BertForSequenceClassification model
# Do not ouput the attentions and all hidden states

config = GPT2Config.from_pretrained('gpt2',
                                    output_attentions=False,
                                    output_hidden_states=False,
                                    num_labels=2)

# Set to 'gpt2' (the smallest GPT2 which is 120 M parameters)
# Use the config above and set other labels as needed
model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=config)

# Set the pad token id to the eos token id
model.config.pad_token_id = model.config.eos_token_id

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5
# See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 5e-5,
    eps = 1e-08
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Set the model to the right device

In [ ]:
# If on GPU, do as below
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = model.to(device)

# Recommended number of epochs: See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

In [ ]:
# Print all the layers of this GPT2 model and the number of parameters per layer
# If this is False, fine tune just the classifier layer and leave all other GPT2 parameters alone
# If this is True, fine tune everything
fine_tune = False

# Turn off gradients using the above
total_parameters = 0

for name, param in model.named_parameters():
    if not fine_tune and 'score' not in name:
        param.requires_grad = False
    print(name, torch.numel(param), param.shape, param.requires_grad)
    total_parameters += torch.numel(param)

assert(total_parameters == 124441344)

transformer.wte.weight 38597376 torch.Size([50257, 768]) False
transformer.wpe.weight 786432 torch.Size([1024, 768]) False
transformer.h.0.ln_1.weight 768 torch.Size([768]) False
transformer.h.0.ln_1.bias 768 torch.Size([768]) False
transformer.h.0.attn.c_attn.weight 1769472 torch.Size([768, 2304]) False
transformer.h.0.attn.c_attn.bias 2304 torch.Size([2304]) False
transformer.h.0.attn.c_proj.weight 589824 torch.Size([768, 768]) False
transformer.h.0.attn.c_proj.bias 768 torch.Size([768]) False
transformer.h.0.ln_2.weight 768 torch.Size([768]) False
transformer.h.0.ln_2.bias 768 torch.Size([768]) False
transformer.h.0.mlp.c_fc.weight 2359296 torch.Size([768, 3072]) False
transformer.h.0.mlp.c_fc.bias 3072 torch.Size([3072]) False
transformer.h.0.mlp.c_proj.weight 2359296 torch.Size([3072, 768]) False
transformer.h.0.mlp.c_proj.bias 768 torch.Size([768]) False
transformer.h.1.ln_1.weight 768 torch.Size([768]) False
transformer.h.1.ln_1.bias 768 torch.Size([768]) False
transformer.h.1.a

### Train the model

In [ ]:
# Use torchmetrics to set up accuracy, recall, precision, and auroc
accuracy = Accuracy(task='binary', num_classes=2).to(device)
recall = Recall(task='binary', num_classes=2).to(device)
precision = Precision(task='binary', num_classes=2).to(device)
auroc = AUROC(task="binary").to('cpu')

In [ ]:
# Main training / validation loop
for _ in trange(epochs, desc = 'Epoch'):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        # Put each element of batch onto the device
        batch = tuple(t.to(device) for t in batch)

        # Unpack the batch
        input_ids, attention_mask, labels = batch

        # Set gradients to zero
        optimizer.zero_grad()

        # Forward pass
        train_output = model(input_ids, attention_mask=attention_mask, labels=labels)

        # Backward pass
        loss = train_output.loss
        loss.backward()
        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)

        # Unpack the batch
        input_ids, attention_mask, labels = batch

        with torch.no_grad():
          # Forward pass
            eval_output = model(input_ids, attention_mask=attention_mask)

        # Calculate validation metrics
        labels = labels.to('cpu')
        predicted_labels = torch.argmax(eval_output.logits.to('cpu'), dim=-1)

        val_accuracy.append(accuracy(predicted_labels, labels).item())
        val_recall.append(recall(predicted_labels, labels).item())
        val_precision.append(precision(predicted_labels, labels).item())
        val_auroc.append(auroc(predicted_labels.to('cpu'), labels.to('cpu')).item())

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
Epoch:  50%|█████     | 1/2 [00:12<00:12, 12.42s/it]


	 - Train loss: 0.4315
	 - Validation Accuracy: 0.8795
	 - Validation Precision: 0.4393
	 - Validation Recall: 0.3326
	 - Validation AUROC: 0.6035



Epoch: 100%|██████████| 2/2 [00:24<00:00, 12.28s/it]


	 - Train loss: 0.3071
	 - Validation Accuracy: 0.9196
	 - Validation Precision: 0.6083
	 - Validation Recall: 0.4540
	 - Validation AUROC: 0.6756



### Test on a specific sentence, see the outcome

In [ ]:
new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Spam' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Ham'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Predicted Class:  Ham


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2834: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


### Questions

Question 1: Run the above by fine tuning GPT2 and the classfier head and by not doing this (using GPT2 as a feature encoder). What is the gap between this? What are the metrics we get in each case?

**Metrics**: Fine-tuning GPT-2 and Classifier Head (`fine_tune = True`):

- Epoch 1:
  - Validation Accuracy: 0.9866
  - Validation Precision: 0.8495
  - Validation Recall: 0.8305
  - Validation AUROC: 0.8702
- Epoch 2:
  - Validation Accuracy: 0.9902
  - Validation Precision: 0.8762
  - Validation Recall: 0.8512
  - Validation AUROC: 0.8817
- Predicted Class: Spam

**Metrics**: Using GPT-2 as a Feature Encoder (`fine_tune = False`):

- Epoch 1:
  - Validation Accuracy: 0.8795
  - Validation Precision: 0.4393
  - Validation Recall: 0.3326
  - Validation AUROC: 0.6035
- Epoch 2:
	- Validation Accuracy: 0.9196
	- Validation Precision: 0.6083
	- Validation Recall: 0.4540
	- Validation AUROC: 0.6756
- Predicted Class: Ham

**Gap Analysis**:
- Accuracy Gap: 0.0706
- Precision Gap: 0.2679
- Recall Gap: 0.3972
- AUROC Gap: 0.2061
- Predicted Class: Spam (Case 1) vs Ham (Case 2)

**Observations**:
1. Performance Gap: Fine-tuning the entire model (`fine_tune = True`, Case 1) significantly outperforms using GPT-2 as a feature encoder (`fine_tune = False`, Case 2) across all metrics. Case 2 struggles to learn meaningful representations because it relies entirely on the frozen GPT-2, which may not produce task-specific embeddings. The frozen transformer cannot adapt to the nuances of spam detection, resulting in a substantial performance gap.

2. Case 1 correctly predicts the input as Spam, aligning with typical spam characteristics (e.g., urgency, monetary reward, and excessive capitalization). On the other hand, case 2 misclassifies the input as Ham, highlighting the limitations of using GPT-2 as a feature encoder for tasks requiring deeper contextual understanding.

3. Training Efficiency: Fine-tuning (`fine_tune = True`) took approximately 40 seconds per epoch. Feature Encoder (`fine_tune = False`) took only 12 seconds per epoch. This shows that using GPT-2 as a feature encoder is significantly faster while maintaining the same performance metrics.

**Conclusion**: Fine-tuning GPT-2 effectively identifies spam characteristics by adapting the entire model to the task, significantly outperforming the feature encoder approach. While using GPT-2 as a feature encoder is faster, it risks misclassifications due to the lack of task-specific adaptation, making it unsuitable for most tasks requiring nuanced understanding.